In [ ]:
#Chain CSV generator: From a list of stock tickers get option chains and save to a CSV file.
#Result: Chains not yet available yet in AlphaVantage. Freezing cells.
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
import os
from dotenv import load_dotenv


# Get the current working directory
current_dir = os.getcwd()

# Construct the absolute path to the .env file three directories up
dotenv_path = os.path.join(current_dir, '..', '..', '..', 'a.env')

# Load the environment variables from the .env file
load_dotenv(dotenv_path=dotenv_path)

vantage_key = os.environ['VANTAGE_KEY']

def get_option_chain(ticker):
    """
    Get option chain information for a given stock ticker.

    Parameters:
    - ticker (str): Stock ticker symbol.

    Returns:
    - df (pd.DataFrame): DataFrame with option chain information.
    """

    # Replace 'YOUR_API_KEY' with your actual Alpha Vantage API key
    api_key = vantage_key
    ts = TimeSeries(key=api_key)

    # Get the option chain data
    option_chain, meta_data = ts.get_daily(symbol=ticker)

    # Extract relevant information and create a DataFrame
    data = []
    for expiration_date, contracts in option_chain.items():
        for contract in contracts:
            data.append({
                'Type': contract['option_type'],
                'Expiration Date': expiration_date,
                'Ticker': ticker,
                'Ask Price': contract['askprice'],
                'Bid Price': contract['bidprice']
            })

    df = pd.DataFrame(data)
    return df

def save_option_chain_to_csv(ticker_list, output_file):
    """
    Fetch option chain information for a list of stock tickers and save to a CSV file.

    Parameters:
    - ticker_list (list): List of stock tickers.
    - output_file (str): Output CSV file name.
    """

    # Create an empty DataFrame to store the results
    all_data = pd.DataFrame()

    for ticker in ticker_list:
        try:
            # Get option chain data for the current ticker
            option_chain_data = get_option_chain(ticker)

            # Append the data to the main DataFrame
            all_data = pd.concat([all_data, option_chain_data], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {str(e)}")

    # Save the DataFrame to a CSV file
    all_data.to_csv(output_file, index=False)
    print(f"Option chain data saved to {output_file}")

# Example usage:
tickers = ['SDS', 'BA', 'META']
output_csv = 'F:\valuation_models\output\option_chain_data.csv'
save_option_chain_to_csv(tickers, output_csv)

In [1]:
#yfinance, chain information is outdated, practice run
import os
import yfinance as yf
import pandas as pd
from dotenv import load_dotenv

def get_option_chain(ticker):
    """
    Get historical option prices for a given stock ticker.

    Parameters:
    - ticker (str): Stock ticker symbol.

    Returns:
    - df (pd.DataFrame): DataFrame with historical option prices.
    """

    # Get the option chain data
    option_chain = yf.Ticker(ticker).options

    # Extract relevant information and create a DataFrame
    data = []
    for expiration_date in option_chain:
        option_data = yf.Ticker(ticker).option_chain(expiration_date)
        calls = option_data.calls
        puts = option_data.puts

        # Append call option data
        if not calls.empty:
            data.extend(calls.to_dict(orient='records'))

        # Append put option data
        if not puts.empty:
            data.extend(puts.to_dict(orient='records'))

    df = pd.DataFrame(data)
    return df

def save_option_chain_to_csv(ticker_list, output_file):
    """
    Fetch option chain information for a list of stock tickers and save to a CSV file.

    Parameters:
    - ticker_list (list): List of stock tickers.
    - output_file (str): Output CSV file name.
    """

    # Create an empty DataFrame to store the results
    all_data = pd.DataFrame()

    for ticker in ticker_list:
        try:
            # Get option chain data for the current ticker
            option_chain_data = get_option_chain(ticker)

            # Append the data to the main DataFrame
            all_data = pd.concat([all_data, option_chain_data], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {str(e)}")

    # Save the DataFrame to a CSV file
    all_data.to_csv(output_file, index=False)
    print(f"Option chain data saved to {output_file}")

# Load the environment variables from the .env file
load_dotenv()

# Example usage:
tickers = ['SNAP']
output_csv = 'option_chain_data.csv'
save_option_chain_to_csv(tickers, output_csv)


Option chain data saved to option_chain_data.csv
